# Handling Missing Data

The difference between data found in many tutorials and data in the real world is that **real-world data is rarely clean and homogeneous**.
In particular, many interesting datasets will have some amount of data missing.
To make matters even more complicated, **different data sources may indicate missing data in different ways.**

In this section, we will discuss some general considerations for missing data, discuss how **Pandas** chooses to represent it, and demonstrate some built-in Pandas tools for handling missing data in Python.
Here and throughout the book, we'll refer to missing data in general as ***null*, *NaN*, or *NA* values**.

## Trade-Offs in Missing Data Conventions

There are a number of schemes that have been developed to indicate the presence of missing data in a table or DataFrame.
Generally, they revolve around one of **two strategies**: using a ***mask*** that globally indicates **missing values, or choosing a *sentinel value*** that indicates a missing entry.

In the masking approach, the mask might be an entirely separate Boolean array, or it may involve appropriation of one bit in the data representation to locally indicate the null status of a value.

In the sentinel approach, the sentinel value could be some **data-specific convention, such as indicating a missing integer value with -9999** or some rare bit pattern, or it could be a more global convention, such as indicating a missing floating-point value with NaN (Not a Number), a special value which is part of the IEEE floating-point specification.

None of these approaches is without trade-offs: use of a **separate mask array requires allocation of an additional Boolean array, which adds overhead in both storage and computation**. A **sentinel value reduces the range of valid values that can be represented**, and may require extra (often non-optimized) logic in CPU and GPU arithmetic. Common special values like NaN are not available for all data types.

As in most cases where no universally optimal choice exists, different languages and systems use different conventions.
For example, the R language uses reserved bit patterns within each data type as sentinel values indicating missing data, while the SciDB system uses an extra byte attached to every cell which indicates a NA state.

## Missing Data in Pandas

The way in which **Pandas handles missing values is constrained by its reliance on the NumPy package**, which does not have a built-in notion of NA values for non-floating-point data types.

**Pandas could have followed R's lead** in specifying bit patterns for each individual data type to indicate nullness, but this approach turns out to be rather unwieldy.
While R contains four basic data types, NumPy supports *far* more than this: for example, **while R has a single integer type, NumPy supports *fourteen* basic integer types once you account for available precisions**, signedness, and endianness of the encoding.
**Reserving a specific bit pattern in all available NumPy types would lead to an unwieldy amount of overhead** in special-casing various operations for various types, likely even requiring a new fork of the NumPy package. Further, for the smaller data types (such as 8-bit integers), sacrificing a bit to use as a mask will significantly reduce the range of values it can represent.

**NumPy does have support for masked arrays – that is, arrays that have a separate Boolean mask array attached for marking data as "good" or "bad."**
Pandas could have derived from this, but the **overhead in both storage, computation, and code maintenance** makes that an unattractive choice.

**With these constraints in mind, Pandas chose to use sentinels for missing data**, and further chose to use two already-existing Python null values: the special floating-point ``NaN`` value, and the Python ``None`` object.
This choice has some side effects, as we will see, but in practice ends up being a good compromise in most cases of interest.

### ``None``: Pythonic missing data

The first sentinel value used by Pandas is ``None``, a Python singleton object that is often used for missing data in Python code.
Because it is a Python object, **``None`` cannot be used in any arbitrary NumPy/Pandas array, but only in arrays with data type ``'object'`` (i.e., arrays of Python objects)**:

In [2]:
import numpy as np 
import pandas as pd 

In [9]:
vals1 = np.array([1,None,3,4])
vals1

array([1, None, 3, 4], dtype=object)

**This ``dtype=object`` means that the best common type representation NumPy could infer for the contents of the array is that they are Python objects**.
While this kind of object array is useful for some purposes, **any operations on the data will be done at the Python level, with much more overhead than the typically fast operations** seen for arrays with native types:

In [7]:
for dtype in ['object', 'int']:
    print("dtype =", dtype)
    %timeit np.arange(1E6, dtype=dtype).sum()
    print()

dtype = object
65.3 ms ± 4.36 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

dtype = int
2.57 ms ± 198 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)



The use of Python objects in an array also means that if you perform aggregations like ``sum()`` or ``min()`` across an array with a ``None`` value, you will generally get an error:

In [14]:
vals1.sum() #No se puede operar en un array tipo Object 

TypeError: unsupported operand type(s) for +: 'int' and 'NoneType'

This reflects the fact that addition between an integer and ``None`` is undefined.

### ``NaN``: Missing numerical data

The other missing data representation, **``NaN``** (acronym for *Not a Number*), is different; **it is a special floating-point value recognized by all systems** that use the standard IEEE floating-point representation:

In [3]:
vals2 = np.array([1,np.nan,3,4])

Notice that NumPy chose a native floating-point type for this array: this means that **unlike the object array from before, this array supports fast operations pushed into compiled code.**
You should be aware that ``NaN`` is a bit like a data virus–it infects any other object it touches.
Regardless of the operation, the result of arithmetic with ``NaN`` will be another ``NaN``:

In [11]:
vals2.dtype

dtype('float64')

In [4]:
1+vals2

array([ 2., nan,  4.,  5.])

In [5]:
0*vals2

array([ 0., nan,  0.,  0.])

In [6]:
0*np.nan

nan

Note that this means that aggregates over the values are well defined (i.e., they don't result in an error) but not always useful:

In [8]:
np.sum(vals2),np.min(vals2),np.max(vals2)

(nan, nan, nan)

NumPy does provide some special aggregations that will ignore these missing values:

In [7]:
np.nansum(vals2), np.nanmax(vals2),np.nanmin(vals2)

(8.0, 4.0, 1.0)

Keep in mind that **``NaN`` is specifically a floating-point value; there is no equivalent NaN value for integers, strings, or other types**.

### NaN and None in Pandas

``NaN`` and ``None`` both have their place, and Pandas is built to handle the two of them nearly interchangeably, converting between them where appropriate:

In [12]:
pd.Series([1,np.nan,2,None])

0    1.0
1    NaN
2    2.0
3    NaN
dtype: float64

In [13]:
pd.Series([1,2,None])

0    1.0
1    2.0
2    NaN
dtype: float64

For types that don't have an available sentinel value, Pandas automatically type-casts when NA values are present.
For example, if we set a value in an integer array to ``np.nan``, it will automatically be upcast to a floating-point type to accommodate the NA:

In [15]:
x = pd.Series(range(2), dtype= int)
x

0    0
1    1
dtype: int32

In [17]:
x[0] = None
x   #Pandas por detrás, redimensiona la serie, al generara un None = Nan 

0    NaN
1    1.0
dtype: float64

Notice that in addition to casting the integer array to floating point, **Pandas automatically converts the ``None`` to a ``NaN`` value**.
(Be aware that there is a proposal to add a native integer NA to Pandas in the future; as of this writing, it has not been included).

While this type of magic may feel a bit hackish compared to the more unified approach to NA values in domain-specific languages like R, the Pandas sentinel/casting approach works quite well in practice and in my experience only rarely causes issues.

The following table lists the upcasting conventions in Pandas when NA values are introduced:

|Typeclass     | Conversion When Storing NAs | NA Sentinel Value      |
|--------------|-----------------------------|------------------------|
| ``floating`` | No change                   | ``np.nan``             |
| ``object``   | No change                   | ``None`` or ``np.nan`` |
| ``integer``  | Cast to ``float64``         | ``np.nan``             |
| ``boolean``  | Cast to ``object``          | ``None`` or ``np.nan`` |

**Keep in mind that in Pandas, string data is always stored with an ``object`` dtype.**

## Operating on Null Values

As we have seen, Pandas treats ``None`` and ``NaN`` as essentially interchangeable for indicating missing or null values.
To facilitate this convention, there are several useful methods for detecting, removing, and replacing null values in Pandas data structures.
They are:

- ``isnull()``: Generate a boolean mask indicating missing values
- ``notnull()``: Opposite of ``isnull()``
- ``dropna()``: Return a filtered version of the data
- ``fillna()``: Return a copy of the data with missing values filled or imputed

We will conclude this section with a brief exploration and demonstration of these routines.

### Detecting null values
Pandas data structures have two useful methods for detecting null data: ``isnull()`` and ``notnull()``.
Either one will return a Boolean mask over the data. For example:

In [21]:
data = pd.Series([1,np.nan, "hello","None"])
data

0        1
1      NaN
2    hello
3     None
dtype: object

In [26]:
data.isnull()

0    False
1     True
2    False
3    False
dtype: bool

In [27]:
#Aplico máscara
data[data.isnull()]

1    NaN
dtype: object

0        1
2    hello
3     None
dtype: object

In [29]:
data[data.notnull()]

0        1
2    hello
3     None
dtype: object

The ``isnull()`` and ``notnull()`` methods produce similar Boolean results for ``DataFrame``s.

### Dropping null values

In addition to the masking used before, there are the convenience methods, ``dropna()``
(which removes NA values) and ``fillna()`` (which fills in NA values). For a ``Series``,
the result is straightforward:

In [30]:
data

0        1
1      NaN
2    hello
3     None
dtype: object

In [31]:
data.dropna()

0        1
2    hello
3     None
dtype: object

For a ``DataFrame``, there are more options.
Consider the following ``DataFrame``:

In [32]:
df = pd.DataFrame([[1,      np.nan, 2],
                   [2,      3,      5],
                   [np.nan, 4,      6]])
df

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


We cannot drop single values from a ``DataFrame``; we can only drop full rows or full columns.
Depending on the application, you might want one or the other, so ``dropna()`` gives a number of options for a ``DataFrame``.

By default, **``dropna()`` will drop all rows** in which *any* null value is present:

In [33]:
df.dropna()

,0,1,2
1,2.0,3.0,5


Alternatively, you can drop NA values along a different axis; ``axis=1`` drops all columns containing a null value:

In [35]:
df.dropna(axis= 1) #También se puede indicar 'columns' = 1 / 'row'= 0 parámetro por defecto.

,2
0,2
1,5
2,6


But this drops some good data as well; **you might rather be interested in dropping rows or columns with *all* NA values, or a majority of NA values.**
This can be specified through the ``how`` or ``thresh`` parameters, which allow fine control of the number of nulls to allow through.

The default is ``how='any'``, such that any row or column (depending on the ``axis`` keyword) containing a null value will be dropped.
You can also specify ``how='all'``, which will only drop rows/columns that are *all* null values:

In [36]:
df[3]= np.nan
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [37]:
df.dropna(axis='columns', how= 'all')

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


For finer-grained control, the ``thresh`` parameter lets you specify a minimum number of non-null values for the row/column to be kept:

In [39]:
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [41]:
df.dropna(thresh=3)

,0,1,2,3
1,2.0,3.0,5,NaN


Here the first and last row have been dropped, because they contain only two non-null values.

### Filling null values

**Sometimes rather than dropping NA values, you'd rather replace them with a valid value.**
This value might be a single number like zero, or it might be some sort of imputation or interpolation from the good values.
You could do this in-place using the ``isnull()`` method as a mask, but because it is such a common operation Pandas provides the ``fillna()`` method, which returns a copy of the array with the null values replaced.

Consider the following ``Series``:

In [42]:
data= pd.Series([1,np.nan, 2, None, 3], index = list('abcde'))
data

a    1.0
b    NaN
c    2.0
d    NaN
e    3.0
dtype: float64

We can fill NA entries with a single value, such as zero:

In [44]:
data

a    1.0
b    NaN
c    2.0
d    NaN
e    3.0
dtype: float64

In [43]:
data.fillna(0)

a    1.0
b    0.0
c    2.0
d    0.0
e    3.0
dtype: float64

We can specify a forward-fill to propagate the previous value forward:

In [46]:
data.fillna(method='ffill') #Rellena con el valor anterior

a    1.0
b    1.0
c    2.0
d    2.0
e    3.0
dtype: float64

Or we can specify a back-fill to propagate the next values backward:

In [48]:
data.fillna(method='bfill') #Rellena el Nan con el valor siguiente

a    1.0
b    2.0
c    2.0
d    3.0
e    3.0
dtype: float64

For ``DataFrame``s, the options are similar, but we can also specify an ``axis`` along which the fills take place:

In [49]:
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [50]:
df.fillna(method= 'ffill')

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,2.0,4.0,6,NaN


In [51]:
df.fillna(method='bfill') #Rellena el Nan con el valor siguiente


,0,1,2,3
0,1.0,3.0,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


Notice that if a previous value is not available during a forward fill, the NA value remains.

# Algunos ejemplos prácticos

In [52]:
import pandas as pd
import numpy as np

In [53]:
dummy_df = pd.DataFrame({'Sno': range(1,10),
                        'Name': ['John', 'Jimmy', 'Felicia', 'Sophia', 'Bob',
                                 'Billy', 'Kate', 'Will', 'Scott'],
                         
                        'Age': [25,26,28,np.nan,np.nan,30,31,29,np.nan],
                         
                        'Height(cm)': [160,163,154,143,np.nan,156,160,np.nan,148]})

In [54]:
dummy_df

,Sno,Name,Age,Height(cm)
0,1,John,25.0,160.0
1,2,Jimmy,26.0,163.0
2,3,Felicia,28.0,154.0
3,4,Sophia,NaN,143.0
4,5,Bob,NaN,NaN
5,6,Billy,30.0,156.0
6,7,Kate,31.0,160.0
7,8,Will,29.0,NaN
8,9,Scott,NaN,148.0


# Reasons for missing Data  
1. User did not want to fill data due to privacy issues  
2. Loss of data while transferring  
3. Insufficient information to fill a particular column, etc.  

There can be multiple reasons for missing values in a dataset

## df.describe()

In [56]:
dummy_df.describe()

,Sno,Age,Height(cm)
count,9.000000,6.000000,7.000000
mean,5.000000,28.166667,154.857143
std,2.738613,2.316607,7.174691
min,1.000000,25.000000,143.000000
25%,3.000000,26.500000,151.000000
50%,5.000000,28.500000,156.000000
75%,7.000000,29.750000,160.000000
max,9.000000,31.000000,163.000000


Here `count` returns Non-null values

In [57]:
dummy_df.count()

Sno           9
Name          9
Age           6
Height(cm)    7
dtype: int64

## Since NaN is considered float, the data is converted to float type

In [58]:
dummy_df.dtypes

Sno             int64
Name           object
Age           float64
Height(cm)    float64
dtype: object

## df.isna() and df.notna()

In [59]:
dummy_df.isna()

,Sno,Name,Age,Height(cm)
0,False,False,False,False
1,False,False,False,False
2,False,False,False,False
3,False,False,True,False
4,False,False,True,True
5,False,False,False,False
6,False,False,False,False
7,False,False,False,True
8,False,False,True,False


In [60]:
dummy_df.notna()

,Sno,Name,Age,Height(cm)
0,True,True,True,True
1,True,True,True,True
2,True,True,True,True
3,True,True,False,True
4,True,True,False,False
5,True,True,True,True
6,True,True,True,True
7,True,True,True,False
8,True,True,False,True


## df.isnull()

In [63]:
dummy_df.isnull()

,Sno,Name,Age,Height(cm)
0,False,False,False,False
1,False,False,False,False
2,False,False,False,False
3,False,False,True,False
4,False,False,True,True
5,False,False,False,False
6,False,False,False,False
7,False,False,False,True
8,False,False,True,False


## Finding count of missing values in each column

In [64]:
dummy_df.isnull().sum()

Sno           0
Name          0
Age           3
Height(cm)    2
dtype: int64

In [65]:
string_dummy_df = pd.DataFrame({
                               'Device_name': ['Synringe', 'Ventilator', 'Surgical Gloves',
                               'Stethescopes', 'Vials container'],
                               'Device_description': ['Used to inject medicine', 'Used to help patients breath',
                                                     np.nan, np.nan, np.nan],
                               'Single-Use': [True, False, True, np.nan, np.nan]},
                              index=range(1,6))

In [66]:
string_dummy_df

,Device_name,Device_description,Single-Use
1,Synringe,Used to inject medicine,True
2,Ventilator,Used to help patients breath,False
3,Surgical Gloves,NaN,True
4,Stethescopes,NaN,NaN
5,Vials container,NaN,NaN


### Sometimes information for a column is insufficient to assign a value

# Missing data for datetime

In [67]:
time_df = pd.DataFrame({'Sno': range(1,10),
                        'Name': ['John', 'Jimmy', 'Felicia', 'Sophia', 'Bob',
                                 'Billy', 'Kate', 'Will', 'Scott'],
                         
                        'Age': [25,26,28,np.nan,np.nan,30,31,29,np.nan],
                         
                        'Height(cm)': [160,163,154,143,np.nan,156,160,np.nan,148],
                       'birthday': ['1994-01-01', np.nan, '1994-01-01', np.nan,'1994-01-01',
                                   '1994-01-01', '1994-01-01', '1994-01-01',np.nan]})

In [69]:
time_df

,Sno,Name,Age,Height(cm),birthday
0,1,John,25.0,160.0,1994-01-01
1,2,Jimmy,26.0,163.0,NaN
2,3,Felicia,28.0,154.0,1994-01-01
3,4,Sophia,NaN,143.0,NaN
4,5,Bob,NaN,NaN,1994-01-01
5,6,Billy,30.0,156.0,1994-01-01
6,7,Kate,31.0,160.0,1994-01-01
7,8,Will,29.0,NaN,1994-01-01
8,9,Scott,NaN,148.0,NaN


In [71]:
type(time_df.birthday[0])

str

In [75]:
time_df.birthday = pd.to_datetime(time_df['birthday'])
type(time_df.birthday[0])

pandas._libs.tslibs.timestamps.Timestamp

In [76]:
time_df

,Sno,Name,Age,Height(cm),birthday
0,1,John,25.0,160.0,1994-01-01
1,2,Jimmy,26.0,163.0,NaT
2,3,Felicia,28.0,154.0,1994-01-01
3,4,Sophia,NaN,143.0,NaT
4,5,Bob,NaN,NaN,1994-01-01
5,6,Billy,30.0,156.0,1994-01-01
6,7,Kate,31.0,160.0,1994-01-01
7,8,Will,29.0,NaN,1994-01-01
8,9,Scott,NaN,148.0,NaT


### In datetime format, missing values are denoted by NaT

### NaN values are considered to be float by default in pandas

# Dealing with missing value

## 1. Ignoring Missing Data

## 1.1 Ignoring rows with missing columns

In [79]:
dummy_df = pd.DataFrame({'Sno': range(1,10),
                        'Name': ['John', 'Jimmy', 'Felicia', 'Sophia', 'Bob',
                                 'Billy', 'Kate', 'Will', 'Scott'],
                         
                        'Age': [25,26,28,np.nan,np.nan,30,31,29,np.nan],
                         
                        'Height(cm)': [160,163,154,143,np.nan,156,160,np.nan,148]})

In [81]:
dummy_df

,Sno,Name,Age,Height(cm)
0,1,John,25.0,160.0
1,2,Jimmy,26.0,163.0
2,3,Felicia,28.0,154.0
3,4,Sophia,NaN,143.0
4,5,Bob,NaN,NaN
5,6,Billy,30.0,156.0
6,7,Kate,31.0,160.0
7,8,Will,29.0,NaN
8,9,Scott,NaN,148.0


In [80]:
dummy_df.dropna()

,Sno,Name,Age,Height(cm)
0,1,John,25.0,160.0
1,2,Jimmy,26.0,163.0
2,3,Felicia,28.0,154.0
5,6,Billy,30.0,156.0
6,7,Kate,31.0,160.0


In [83]:
remove_nas_df = dummy_df.dropna()
remove_nas_df

,Sno,Name,Age,Height(cm)
0,1,John,25.0,160.0
1,2,Jimmy,26.0,163.0
2,3,Felicia,28.0,154.0
5,6,Billy,30.0,156.0
6,7,Kate,31.0,160.0


## Not a good idea since we have deleted 4 out of 9 rows (~44% data)

## 1.2 Removing rows with majority columns as missing

In [97]:
dummy_majority_df = pd.DataFrame({
                        'Name': ['John', 'Jimmy', 'Felicia', 'Sophia', 'Bob',
                                 'Billy', 'Kate', 'Will', 'Scott'],
                         
                        'Age': [25,26,28,np.nan,np.nan,30,31,29,np.nan],
                         
                        'Height(cm)': [160,163,154,143,np.nan,156,160,np.nan,148],
                        'Marks(100)': [80,np.nan,np.nan,np.nan,np.nan,np.nan,
                                      np.nan,np.nan,np.nan],
                        'Country': ['USA', 'UK', 'USA', np.nan, np.nan, 'France',
                                   'Italy', 'Russia', np.nan],
                        'City': ['New York', 'London', 'Miami', np.nan, np.nan,
                                'Paris', 'Rome', 'Moscow', np.nan]})

In [98]:
dummy_majority_df

,Name,Age,Height(cm),Marks(100),Country,City
0,John,25.0,160.0,80.0,USA,New York
1,Jimmy,26.0,163.0,NaN,UK,London
2,Felicia,28.0,154.0,NaN,USA,Miami
3,Sophia,NaN,143.0,NaN,NaN,NaN
4,Bob,NaN,NaN,NaN,NaN,NaN
5,Billy,30.0,156.0,NaN,France,Paris
6,Kate,31.0,160.0,NaN,Italy,Rome
7,Will,29.0,NaN,NaN,Russia,Moscow
8,Scott,NaN,148.0,NaN,NaN,NaN


#### `thresh=x` attribute tells df.dropna() to keep rows with atleast 'x' Non-Null values

In [99]:
dummy_majority_df.dropna(thresh=4)

,Name,Age,Height(cm),Marks(100),Country,City
0,John,25.0,160.0,80.0,USA,New York
1,Jimmy,26.0,163.0,NaN,UK,London
2,Felicia,28.0,154.0,NaN,USA,Miami
5,Billy,30.0,156.0,NaN,France,Paris
6,Kate,31.0,160.0,NaN,Italy,Rome
7,Will,29.0,NaN,NaN,Russia,Moscow


In [100]:
dummy_majority_df.dropna(thresh=6)

,Name,Age,Height(cm),Marks(100),Country,City
0,John,25.0,160.0,80.0,USA,New York


### Dropping columns by percentage of missing values   
Drop columns with say over 40% empty values

**Note** : axis=0 is for rows and axis=1 is for columns

In [101]:
dummy_majority_df.dropna(axis=1,thresh=int(len(dummy_majority_df)*0.4))

,Name,Age,Height(cm),Country,City
0,John,25.0,160.0,USA,New York
1,Jimmy,26.0,163.0,UK,London
2,Felicia,28.0,154.0,USA,Miami
3,Sophia,NaN,143.0,NaN,NaN
4,Bob,NaN,NaN,NaN,NaN
5,Billy,30.0,156.0,France,Paris
6,Kate,31.0,160.0,Italy,Rome
7,Will,29.0,NaN,Russia,Moscow
8,Scott,NaN,148.0,NaN,NaN


`Marks` column is deleted since it had more than 40% empty values

### Dropping rows with majority columns as null values

Keep rows with atleast 60% of values filled

In [103]:
dummy_majority_df.dropna(axis= 0, thresh=int(0.6*len(dummy_majority_df.columns)))

,Name,Age,Height(cm),Marks(100),Country,City
0,John,25.0,160.0,80.0,USA,New York
1,Jimmy,26.0,163.0,NaN,UK,London
2,Felicia,28.0,154.0,NaN,USA,Miami
5,Billy,30.0,156.0,NaN,France,Paris
6,Kate,31.0,160.0,NaN,Italy,Rome
7,Will,29.0,NaN,NaN,Russia,Moscow


## 2. Imputing Values

## 2.1 Filling with generic values

In [114]:
dummy_df = pd.DataFrame({
                        'Name': ['John', 'Jimmy', 'Felicia', 'Sophia', 'Bob',
                                 'Billy', 'Kate', 'Will', 'Scott'],
                         
                        'Age': [25,26,28,np.nan,np.nan,30,31,29,np.nan],
                         
                        'Height(cm)': [160,163,154,143,np.nan,156,160,np.nan,148]})

In [106]:
dummy_df

,Name,Age,Height(cm)
0,John,25.0,160.0
1,Jimmy,26.0,163.0
2,Felicia,28.0,154.0
3,Sophia,NaN,143.0
4,Bob,NaN,NaN
5,Billy,30.0,156.0
6,Kate,31.0,160.0
7,Will,29.0,NaN
8,Scott,NaN,148.0


In [110]:
dummy_df.fillna(-1)

,Name,Age,Height(cm)
0,John,25.0,160.0
1,Jimmy,26.0,163.0
2,Felicia,28.0,154.0
3,Sophia,-1.0,143.0
4,Bob,-1.0,-1.0
5,Billy,30.0,156.0
6,Kate,31.0,160.0
7,Will,29.0,-1.0
8,Scott,-1.0,148.0


**Note** changes will be permanently reflected only when inplace=True

In [113]:
dummy_df.fillna(dummy_df.mean(), inplace = True)
dummy_df

,Name,Age,Height(cm)
0,John,25.000000,160.000000
1,Jimmy,26.000000,163.000000
2,Felicia,28.000000,154.000000
3,Sophia,28.166667,143.000000
4,Bob,28.166667,154.857143
5,Billy,30.000000,156.000000
6,Kate,31.000000,160.000000
7,Will,29.000000,154.857143
8,Scott,28.166667,148.000000


In [115]:
dummy_df.bfill()

,Name,Age,Height(cm)
0,John,25.0,160.0
1,Jimmy,26.0,163.0
2,Felicia,28.0,154.0
3,Sophia,30.0,143.0
4,Bob,30.0,156.0
5,Billy,30.0,156.0
6,Kate,31.0,160.0
7,Will,29.0,148.0
8,Scott,NaN,148.0


In [116]:
dummy_df.ffill()

,Name,Age,Height(cm)
0,John,25.0,160.0
1,Jimmy,26.0,163.0
2,Felicia,28.0,154.0
3,Sophia,28.0,143.0
4,Bob,28.0,143.0
5,Billy,30.0,156.0
6,Kate,31.0,160.0
7,Will,29.0,160.0
8,Scott,29.0,148.0


In [118]:
dummy_df.fillna(method = 'ffill')

,Name,Age,Height(cm)
0,John,25.0,160.0
1,Jimmy,26.0,163.0
2,Felicia,28.0,154.0
3,Sophia,28.0,143.0
4,Bob,28.0,143.0
5,Billy,30.0,156.0
6,Kate,31.0,160.0
7,Will,29.0,160.0
8,Scott,29.0,148.0


## 2.2 Filling with central tendencies

In [121]:
mean_age = dummy_df['Age'].mean()
mean_height= dummy_df['Height(cm)'].mean()

In [122]:
map_dict = {'Age':mean_age,'Height(cm)':mean_height}
map_dict

{'Age': 28.166666666666668, 'Height(cm)': 154.85714285714286}

In [123]:
dummy_df.fillna(map_dict)

,Name,Age,Height(cm)
0,John,25.000000,160.000000
1,Jimmy,26.000000,163.000000
2,Felicia,28.000000,154.000000
3,Sophia,28.166667,143.000000
4,Bob,28.166667,154.857143
5,Billy,30.000000,156.000000
6,Kate,31.000000,160.000000
7,Will,29.000000,154.857143
8,Scott,28.166667,148.000000


## 2.3 Imputing values based on condition

In [124]:
weight_df = pd.DataFrame({
                        'Gender': ['Male', 'Female', 'Male', 'Female', 'Female',
                                 'Male', 'Female', 'Female', 'Male', 'Male', 'Male', 'Female'],
                         
                        'Weight(kg)': [70, 55, 65, np.nan, 60, np.nan, 52, 53, 85, 75, np.nan, 68]})

In [125]:
weight_df

,Gender,Weight(kg)
0,Male,70.0
1,Female,55.0
2,Male,65.0
3,Female,NaN
4,Female,60.0
5,Male,NaN
6,Female,52.0
7,Female,53.0
8,Male,85.0
9,Male,75.0


In [126]:
weight_df.groupby('Gender').transform(lambda x: x.fillna(x.mean()))

,Weight(kg)
0,70.00
1,55.00
2,65.00
3,57.60
4,60.00
5,73.75
6,52.00
7,53.00
8,85.00
9,75.00
